### Researched Quantities

- Building this model based on a 10-year rainstorm, which is 5.5 inches in 24 hours.
-Here is the link to the sheet with the data about the plants https://1drv.ms/x/c/348b84fad346e9f1/EX-eKXt_eg1HnNybvuDh2aUBIBjV-u4fKmu33ILq_tpZVw?e=BLTTLG 
- Unit conversions:
    - 1 square mile = 640 acres
    - 1 square mile = 27,878,400 square feet
    - 1 foot = 12 inches
    - 1 gallon = 0.13368 cubic feet
- According to HomeAdvisor, it costs between $50 to $250 per linear foot to insall a sewer line. I feel like since it's NYC it'll be more expensive but it's so much that they can probably get pretty good pricing so I'm just going to guess $150 per linear foot.
    https://www.homeadvisor.com/cost/plumbing/install-a-sewer-main/
- according to NYC waterboards, the sewer rate $7.14 / ft^3 = $954479.17 $/ MG in 2024: https://www.nyc.gov/site/nycwaterboard/rates/rates-regulations.page

In [16]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

using JuMP
using HiGHS
using DataFrames
using GraphRecipes
using Plots
using Measures
using MarkdownTables

  Activating project at `c:\Users\arisc\OneDrive\Desktop\BEE 4750\4750project`


In [17]:
#deriving objective function
facilities = ["26th Ward", "Bowery Bay", "Coney Island", "Hunts Point", "Jamaica", "Newtown Creek", "North River", "Oakwood Beach", "Rockaway", "Owls Head", "Wards Island", "Tallman Island", "Port Richmond", "Red Hook"]
design_capacity = [85, 150, 110, 200, 100, 310, 170, 39.9, 45, 120, 275, 80, 60, 60] #MGD
population_served = [283428, 848328, 596326, 684569, 728123, 1068012, 588772, 244918, 90474, 758007, 1061558, 410812, 198128, 192050] #
sanitary_demand = 80*(10^-6)*population_served #MGD = (gal/(day * person)) * people * 10^-6 MGD/(gal/d)


cost_per_mile = 150*5280 #$/ft * ft/mile
distance = [10.25, 9.55, 10, 11, 13, 5.7, 9.7, 11.25, 14.25, 4.45, 8.75, 12.6, 6.15, 3.7] #miles
W_coef = distance*cost_per_mile

storm_height = 5.5/12 #feet
drainage_area = [5907, 15203, 15087, 16664, 25313, 15656, 6030, 10779, 6259, 12947, 12056, 16860, 9665, 3200] #acres = (miles^2)/640
storm_vol = (storm_height * drainage_area)/3.06888 #MGD = ((in/day)*acre)/(3.06888 acre foot / Mgal)
treatment_cost = 954479.17 #dollars / MGD

CSO_cost = 10^10 #dollars (aka 10 billion dollars)


10000000000

In [38]:
#Setting Up Optimization
stormwater_model = Model(HiGHS.Optimizer) #initialize model object

#create variables: amount of wastewater diverted to new plant
@variable(stormwater_model, W[1:14], Bin) #will this plant overflow in a 10-year storm? - boolean
@variable(stormwater_model, Q[1:14] >= 0) #how much water will go to the new plant
@variable(stormwater_model, O[1:14], Bin) #will we get an overflow in a 10-year storm

#create objective function
@objective(stormwater_model, Min, sum(W_coef .* W) + sum(treatment_cost .* Q) + sum(CSO_cost .* O))

#constraints
#cso_name = ["CSO1", "CSO2", "CSO3", "CSO4", "CSO5", "CSO6", "CSO7", "CSO8", "CSO9", "CSO10", "CSO11", "CSO12", "CSO13", "CSO14"]
@constraint(stormwater_model, overflow1, O[1] => {storm_vol[1] .+ sanitary_demand[1] <= design_capacity[1]})
@constraint(stormwater_model, overflow2, O[2] => {storm_vol[2] .+ sanitary_demand[2] <= design_capacity[2]})
@constraint(stormwater_model, overflow3, O[3] => {storm_vol[3] .+ sanitary_demand[3] <= design_capacity[3]})
@constraint(stormwater_model, overflow4, O[4] => {storm_vol[4] .+ sanitary_demand[4] <= design_capacity[4]})
@constraint(stormwater_model, overflow5, O[5] => {storm_vol[5] .+ sanitary_demand[5] <= design_capacity[5]})
@constraint(stormwater_model, overflow6, O[6] => {storm_vol[6] .+ sanitary_demand[6] <= design_capacity[6]})
@constraint(stormwater_model, overflow7, O[7] => {storm_vol[7] .+ sanitary_demand[7] <= design_capacity[7]})
@constraint(stormwater_model, overflow8, O[8] => {storm_vol[8] .+ sanitary_demand[8] <= design_capacity[8]})
@constraint(stormwater_model, overflow9, O[9] => {storm_vol[9] .+ sanitary_demand[9] <= design_capacity[9]})
@constraint(stormwater_model, overflow10, O[10] => {storm_vol[10] .+ sanitary_demand[10] <= design_capacity[10]})
@constraint(stormwater_model, overflow11, O[11] => {storm_vol[11] .+ sanitary_demand[11] <= design_capacity[11]})
@constraint(stormwater_model, overflow12, O[12] => {storm_vol[12] .+ sanitary_demand[12] <= design_capacity[12]})
@constraint(stormwater_model, overflow13, O[13] => {storm_vol[13] .+ sanitary_demand[13] <= design_capacity[13]})
@constraint(stormwater_model, overflow14, O[14] => {storm_vol[14] .+ sanitary_demand[14] <= design_capacity[14]})

#for i in 1:14
#    @constraint(stormwater_model, O[i] => {storm_vol[i] .+ sanitary_demand[i] <= design_capacity[i]})
#end

overflow14 : O[14] --> {0 <= -433.2799389310323}

In [39]:
#optimize
optimize!(stormwater_model)
@show value.(W)
@show value.(overflow1)
@show value.(overflow2)
@show value.(overflow3)
@show value.(overflow4)
@show value.(overflow5)
@show value.(overflow6)
@show value.(overflow7)
@show value.(overflow8)
@show value.(overflow9)
@show value.(overflow10)
@show value.(overflow11)
@show value.(overflow12)
@show value.(overflow13)
@show value.(overflow14)

Running HiGHS 1.8.1 (git hash: 4a7f24ac6): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 4e+03]
  Cost   [1e+06, 1e+10]
  Bound  [1e+00, 1e+00]
  RHS    [4e+02, 4e+03]
Presolving model
0 rows, 0 cols, 0 nonzeros  0s
0 rows, 0 cols, 0 nonzeros  0s
Presolve: Optimal

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic; L => Sub-MIP;
     P => Empty MIP; R => Randomized rounding; S => Solve LP; T => Evaluate node; U => Unbounded;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   0               0                  0.00%        0      0      0         0     0.0s

Solving report
  Status            Optimal
  Pr

ArgumentError: ArgumentError: Bridge of type `IndicatorToMILPBridge` does not support accessing the attribute `MathOptInterface.ConstraintPrimal(1)`. If you encountered this error unexpectedly, it probably means your model has been reformulated using the bridge, and you are attempting to query an attribute that we haven't implemented yet for this bridge. Please open an issue at https://github.com/jump-dev/MathOptInterface.jl/issues/new and provide a reproducible example explaining what you were trying to do.